In [69]:
# import required modules

from bs4 import BeautifulSoup
import requests
from pprint import pprint
import csv

In [9]:
# take the url and make a BeautifulSoup object

url = "https://www.payscale.com/college-salary-report/majors-that-pay-you-back/bachelors"

response = requests.get(url)
# response.raise_for_status
# <bound method Response.raise_for_status of <Response [200]>>
soup = BeautifulSoup(response.text, "html.parser")

In [35]:
# get the headers as the first column
headers = [i.text for i in soup.find_all(class_="data-table__header")]
print(headers)

['Rank', 'Major', 'Degree Type', 'Early Career Pay', 'Mid-Career Pay', '% High Meaning']


In [17]:
# make a list of lists, or dict of dicts
"""list = [[1, "petrol eng", "bachelors", "$93,200", "$187,300", "%67"],[2,...]]
"""

""" dict = {
    1: {
        major: "Petrol Eng.",
        degree_type: "bachelors",
        early_pay: "$93,200",
        mid_pay: "$187,300",
        migh_meaning: "%67"
    },
    2:{
        .
        .
        .
    }
}"""


# figure out how to fill each column with required info
all_rows = soup.find_all(class_="data-table__row")
first_row = all_rows[0]


In [34]:
# write a function for each row to fill correspondingly

# go 1: list of lists
# pprint(first_row.find_all(class_="data-table__value"))  # gives intended result

def make_row(row: list) -> list:
    """ returns each cell's text as strings in a list """
    return [i.text for i in row.find_all(class_="data-table__value")]

first_info = [i.text for i in first_row.find_all(class_="data-table__value")]
print(first_info)


['1', 'Petroleum Engineering', 'Bachelors', '$93,200', '$187,300', '67%']


In [49]:
# write a function that takes an entire page and make a list of lists filled with info

def all_page_row(all_rows: list) -> list:
    """takes a list filled with soup objects for each page... 
    class_="data-table__row" must be defined before passing through the function!!! """
    return [make_row(each_row) for each_row in all_rows]


# build the first page of the database
database = [headers] + all_page_row(all_rows)

print(len(database))

    

26


In [57]:
# first page done...
# write a script that goes through all pages with try..except


for i in range(2,5):
    next_url = f"https://www.payscale.com/college-salary-report/majors-that-pay-you-back/bachelors/page/{i}"
    response = requests.get(next_url)

    if "200" in str(response.raise_for_status):
        soup = BeautifulSoup(response.text, "html.parser")
        current_rows = soup.find_all(class_="data-table__row")
        database += all_page_row(current_rows)

print(len(database))



101


In [70]:
# make a csv file that fits the example
# pprint(database)

with open("salaries_updated.csv", "w") as df:
    writer = csv.writer(df)
    writer.writerows(database)